In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta

# Define API endpoints and parameters
octopus_energy_api_url = 'https://api.octopus.energy/v1/products/'
carbon_intensity_api_url = 'https://api.carbonintensity.org.uk/intensity/date'
google_api_url = 'https://maps.googleapis.com/maps/api/'

# Define the date range for the data
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 1, 7)

# Initialize empty lists to store data
timestamps = []
energy_consumption = []
renewable_generation = []
storage_capacity = []
storage_charge_rate = []
storage_discharge_rate = []
carbon_intensity = []

# Generate a date range with hourly intervals
date_range = pd.date_range(start=start_date, end=end_date, freq='H')

def fetch_carbon_intensity(date):
    response = requests.get(f"{carbon_intensity_api_url}/{date.strftime('%Y-%m-%d')}")
    data = response.json()
    if 'data' in data:
        intensity_data = data['data']
        actual_intensity = intensity_data[0]['intensity']['actual']  # Assuming you want the actual intensity value
        return actual_intensity
    else:
        return None  # Handle error cases or missing data


# Example function to fetch data from the Octopus Energy API
def fetch_octopus_energy_data():
    # This function should include API key and actual data fetching logic
    pass

# Example function to fetch Google API data
def fetch_google_api_data():
    # This function should include API key and actual data fetching logic
    pass

# Populate the lists with synthetic or fetched data
for timestamp in date_range:
    timestamps.append(timestamp)
    energy_consumption.append(100 + (50 * timestamp.hour % 24))
    renewable_generation.append(200 + (30 * timestamp.hour % 24))
    storage_capacity.append(500)
    storage_charge_rate.append(25)
    storage_discharge_rate.append(25)
    carbon_intensity.append(fetch_carbon_intensity(timestamp))


# Create a DataFrame
data = pd.DataFrame({
    'timestamp': timestamps,
    'energy_consumption': energy_consumption,
    'renewable_generation': renewable_generation,
    'storage_capacity': storage_capacity,
    'storage_charge_rate': storage_charge_rate,
    'storage_discharge_rate': storage_discharge_rate,
    'carbon_intensity': carbon_intensity
})

# Convert 'timestamp' column to numeric if needed (usually it's datetime, which is fine)
# data['timestamp'] = data['timestamp'].astype('int64')  # Example conversion if necessary

# Save to CSV
data.to_csv('energy_data.csv', index=False)



In [3]:
df = pd.read_csv('energy_data.csv')

print(df.to_string())

               timestamp  energy_consumption  renewable_generation  storage_capacity  storage_charge_rate  storage_discharge_rate  carbon_intensity
0    2023-01-01 00:00:00                 100                   200               500                   25                      25                72
1    2023-01-01 01:00:00                 102                   206               500                   25                      25                72
2    2023-01-01 02:00:00                 104                   212               500                   25                      25                72
3    2023-01-01 03:00:00                 106                   218               500                   25                      25                72
4    2023-01-01 04:00:00                 108                   200               500                   25                      25                72
5    2023-01-01 05:00:00                 110                   206               500                   25       

In [4]:
!pip install scikeras[tensorflow]
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from scikeras.wrappers import KerasRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_absolute_error
from flask import Flask, request, jsonify

# Load data
data = pd.read_csv('energy_data.csv')

# Preprocess data
data.fillna(method='ffill', inplace=True)
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Feature engineering
data['hour'] = data.index.hour
data['day'] = data.index.dayofweek

# Split data
X = data.drop('carbon_intensity', axis=1)
y = data['carbon_intensity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define LSTM model
def create_lstm_model():
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define CNN model
def create_cnn_model():
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Define DNN model
def create_dnn_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

# Wrapping models for ensemble
lstm_model = KerasRegressor(build_fn=create_lstm_model, epochs=50, batch_size=32, verbose=0)
cnn_model = KerasRegressor(build_fn=create_cnn_model, epochs=50, batch_size=32, verbose=0)
dnn_model = KerasRegressor(build_fn=create_dnn_model, epochs=50, batch_size=32, verbose=0)

# Ensemble Model
ensemble = VotingRegressor(estimators=[('lstm', lstm_model), ('cnn', cnn_model), ('dnn', dnn_model)])
ensemble.fit(X_train, y_train)

# Evaluation
y_pred = ensemble.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

test_input = X_test[0].reshape(1, -1)
predicted_carbon_savings = ensemble.predict(test_input)
print(f'Predicted Carbon Savings: {predicted_carbon_savings[0]}')


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the

Mean Absolute Error: 19.646112507787244


Predicted Carbon Savings: 86.43097686767578
